# Convolutional Sentiment Analysis

- Dataset: Moview review (rt-polarity.neg, rt-polarity.pos)
    - Source: https://github.com/yoonkim/CNN_sentence
    
- Code reference code
    - https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

- Using kaggle kernel
- Pytorch

**> Reference paper**
- Convolutional Neural Networks for Sentence Classification (EMNLP 2014)
- https://arxiv.org/abs/1408.5882

## import modules

In [ ]:
import numpy as np
from collections import defaultdict
import sys, re
import pandas as pd
import os
import pickle
import codecs
import random
import json
import torch
# from torchtext import data
from torchtext.data import TabularDataset, Field, LabelField, BucketIterator
# from torchtext import datasets

from tqdm import tqdm_notebook

## preprocess and load data

In [1]:
## preprocess from original source code: https://github.com/yoonkim/CNN_sentence

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [ ]:
def build_data(data_dir = '/kaggle/input/polaritydata/', cv=10):
  
    revs = []
    pos = codecs.open(os.path.join(data_dir, "rt-polarity.pos"), "r", encoding='utf-8', errors='ignore').read()
    neg = codecs.open(os.path.join(data_dir, "rt-polarity.neg"), "r", encoding='utf-8', errors='ignore').read()
    pos_list = [clean_str(sent) for sent in pos.split('\n')[:-1]]
    neg_list = [clean_str(sent) for sent in neg.split('\n')[:-1]]
    print('pos len', len(pos_list))
    print('neg len', len(neg_list))
    print('total len',len(pos_list) + len(neg_list) )
  
    for sent in pos_list:
        datum = {'label': 1,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                 }
        revs.append(datum)

    for sent in neg_list:
        datum = {'label': 0,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                }
        revs.append(datum)

    word_to_idx = {'@pad': 0}

    for sent in pos_list + neg_list:
        for word  in sent.split():
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)



    print(f'num of total data: {len(revs)}')
    print(f'num of vocab: {len(word_to_idx)}')


    df = pd.DataFrame(revs) 
    df.to_csv('./polarity_df.csv', index = False, encoding = 'utf-8')
    

    print('save the data')
    
    return revs, word_to_idx

In [ ]:
revs, word_to_idx = build_data()

In [ ]:
# same seed with paper

SEED = 3435

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
TEXT = Field(sequential = True, # text: sequential data
             tokenize = str.split, 
             batch_first = True, 
             fix_length = 56, # padding size: max length of data text
             lower = True)
LABEL = LabelField(sequential = False,
                   dtype = torch.float)

In [ ]:
dataset = TabularDataset(path='./polarity_df.csv', skip_header = True,
    format='csv', fields=[('label', LABEL), ('text', TEXT)]
                        )
# train, test split
train_data, test_data = dataset.split(random_state = random.seed(SEED),split_ratio=0.9)
print('train_data', len(train_data))
print('test_data', len(test_data))

In [ ]:
# show a example
print(vars(train_data[0]))

## loading pretrained word2vec embedding

In [ ]:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('../input/word2vec/GoogleNews-vectors-negative300.bin', binary = True)

In [ ]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)
print(len(TEXT.vocab))
print(LABEL.vocab.stoi)

In [ ]:
TEXT.vocab.vectors.shape

In [ ]:
# for pretrained embedding vectors

EMBEDDING_DIM = 300
w2v_vectors = []
for token, idx in tqdm_notebook(TEXT.vocab.stoi.items()):
    # if word in word2vec vocab -> replace with pretrained word2vec
    if token in w2v.wv.vocab.keys():
        w2v_vectors.append(torch.FloatTensor(w2v[token]))
    # oov -> zero vector
    else:
        w2v_vectors.append(torch.zeros(EMBEDDING_DIM))
#         w2v_vectors.append(torch.distributions.Uniform(-0.25, +0.25).sample((EMBEDDING_DIM,)))
print(len(w2v_vectors))

In [ ]:
print('unk token')
print(w2v_vectors[0])
print('general words')
print(w2v_vectors[10])

In [ ]:
# gpu setting
BATCH_SIZE = 50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# making iterators
train_iterator,  test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle = True)

In [ ]:
for a , b in train_iterator:
    print(a)
    print(a.size())
    print()
    print(b)
    print(b.size())
    break

## model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


## CNN-1D modeling

modeling source code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb


In [ ]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.tanh(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
print('INPUT_DIM', INPUT_DIM)
print('EMBEDDING_DIM', EMBEDDING_DIM)
print('PAD_IDX', PAD_IDX)
print('UNK_IDX', UNK_IDX)

In [ ]:
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
# for check up (before applying word2vec)
test = 'good'
test_indx = TEXT.vocab.stoi[test]
print(f'test string; {test} \nTest string: {test_indx}')

original_vector = w2v[test]
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print(f'before apply\n origianl: {original_vector[:3]} \n torch_vector: {torch_model_vector[:3]}')

In [ ]:
# apply w2v
TEXT.vocab.set_vectors(TEXT.vocab.stoi, w2v_vectors, EMBEDDING_DIM)
pretrained_embeddings = torch.FloatTensor(TEXT.vocab.vectors)
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
# for check up (after applying word2vec)
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print(f'after apply\n origianl: {original_vector[:3]} \n torch_vector: {torch_model_vector[:3]}')

In [ ]:
# padding -> zero vectors
model.embedding.weight.data[PAD_IDX]= torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX]

In [ ]:
# unknown token -> randomly initialized with uniform distribution
model.embedding.weight.data[UNK_IDX] = torch.distributions.Uniform(-0.25, +0.25).sample((EMBEDDING_DIM,))
model.embedding.weight.data[UNK_IDX]

In [ ]:
import torch.optim as optim

# optimizer = optim.Adadelta(model.parameters(), weight_decay=0.95, rho=0.95,eps=1e-6)
optimizer = optim.Adadelta(model.parameters())

# BCEWithLogitsLoss automatically does softmax function
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
    #  l2 norm (weight contraints): 3
        with torch.no_grad():
            for param in model.parameters():
                param.clamp_( min=-3, max = 3)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Test. Loss: {test_loss:.3f} |  Val. Acc: {test_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')